In [1]:
import pandas as pd
import numpy as np
import re
from time import time
import math
import feather
from datetime import datetime
import os
import glob
# from tqdm import tqdm
import optuna
from optuna.integration import KerasPruningCallback
import tempfile
import tensorflow as tf
import random as rn

#Using keras
import keras
from keras import backend as K
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

#from scripts.utils import load_rnn_data
pd.options.display.max_columns = 200

# validation
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, log_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

# others
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


## Load Data

In [2]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(geo_grs):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [3]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [4]:
# load data as x and y of RNN
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [5]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


## Modeling
see https://github.com/pfnet/optuna/blob/master/examples/pruning/keras_integration.py

In [6]:
BATCHSIZE = 128
EPOCHS = 200

In [26]:
def create_model(trial, x_train, y_train):
    # We optimize the number of layers, hidden units and dropout in each layer and
    # the learning rate of RMSProp optimizer.
    
    
    # We define our MLP.
#     n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(x_train.shape[1], x_train.shape[2])))
    #Hitting the first error above - how do we get the time series code to feed an example of 
#     for i in range(n_layers):
#         num_hidden = int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, BATCHSIZE))
#         model.add(Dense(num_hidden, activation='sigmoid'))
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
    model.add(Dropout(rate=dropout))
    model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=lr),
                  metrics=['acc'])

    return model

In [27]:
## Need to define this to produce a training and test sample
def splitting(x_all, y_all):
    # create train and test set
    split = int(x_all.shape[0] * 0.8)   
    x_train = x_all[:split]
    y_train = y_all[:split]
    x_test = x_all[split:]
    y_test = y_all[split:]
    return x_train, y_train, x_test, y_test

In [28]:
#split = int(len(x_all) * 0.8)    
#x_train = x_all[:split]
#y_train = x_all[:split]
#x_test = x_all[split:]
#y_test = x_all[split:]

In [29]:
def objective(trial):
    # Clear clutter form previous session graphs.
    keras.backend.clear_session()
    
    # time series split
    x_train, y_train, x_test, y_test = splitting(x_all, y_all) 
    # Generate our trial model.
    model = create_model(trial, x_train, y_train)

    earlystopping = EarlyStopping(monitor="val_loss", patience=5)
    
    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    model.fit(x_train,
              y_train,
              batch_size=BATCHSIZE,
              callbacks=[KerasPruningCallback(trial, 'val_acc'), earlystopping],
              epochs=EPOCHS,
              validation_split=0.2,
              shuffle=False,
              verbose=1)
    # Evaluate the model accuracy on the test set.
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[0]

In [30]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.MedianPruner()
                           )
study.optimize(objective, n_trials=100)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print('Study statistics: ')
print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of complete trials: ', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print('  Value: ', trials.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Train on 11904 samples, validate on 2976 samples
Epoch 1/200
 4480/11904 [==========>...................] - ETA: 1:41 - loss: 0.6915 - acc: 0.539 - ETA: 1:01 - loss: 0.6521 - acc: 0.613 - ETA: 47s - loss: 0.6120 - acc: 0.681 - ETA: 40s - loss: 0.5752 - acc: 0.73 - ETA: 36s - loss: 0.5449 - acc: 0.76 - ETA: 33s - loss: 0.5188 - acc: 0.78 - ETA: 31s - loss: 0.4970 - acc: 0.79 - ETA: 29s - loss: 0.4783 - acc: 0.80 - ETA: 28s - loss: 0.4629 - acc: 0.81 - ETA: 27s - loss: 0.4497 - acc: 0.82 - ETA: 26s - loss: 0.4389 - acc: 0.82 - ETA: 25s - loss: 0.4310 - acc: 0.83 - ETA: 24s - loss: 0.4242 - acc: 0.83 - ETA: 23s - loss: 0.4192 - acc: 0.84 - ETA: 23s - loss: 0.4141 - acc: 0.84 - ETA: 22s - loss: 0.4088 - acc: 0.84 - ETA: 22s - loss: 0.4049 - acc: 0.84 - ETA: 21s - loss: 0.4012 - acc: 0.85 - ETA: 21s - loss: 0.3973 - acc: 0.85 - ETA: 20s - loss: 0.3937 - acc: 0.85 - ETA: 20s - loss: 0.3904 - acc: 0.85 - ETA: 19s - loss: 0.3865 - acc: 0.85 - ETA: 19s - loss: 0.3769 - acc: 0.86 - ETA: 19s - lo

KeyboardInterrupt: 